# Config

In [1]:
db_name = "energy"
db_user = "dawki"
db_pword_file = "C:\\Users\\dawki\\OneDrive\\Archive\\2019\\Documents\\private.txt"

# Imports

In [2]:
import pandas as pd
import psycopg2
from psycopg2 import DatabaseError
import dateutil.parser
import logging
import datetime as dt
import requests
from DatabaseConnection import DatabaseConnection

In [3]:
from data import ZONE_KEY_TO_REGION, \
REGION_LONG_NAMES, \
ZONE_KEY_TO_NETWORK, \
NETWORK_LONG_NAMES, \
EXCHANGE_MAPPING_DICTIONARY, \
OPENNEM_PRODUCTION_CATEGORIES, \
OPENNEM_STORAGE_CATEGORIES, \
SOURCE

REGION_TO_ZONE_KEY = {value:key for key, value in ZONE_KEY_TO_REGION.items()}

In [4]:
#logger = logging.getLogger(__name__)

# Request data from OpenNEM

In [5]:
region = ZONE_KEY_TO_REGION['AUS-NSW']
month_start = dt.datetime.today().date().replace(day=1)
url = f'http://api.opennem.org.au/stats/power/network/fueltech/NEM/{region}?month={month_start}'

In [6]:
#logger.info(f'Requesting {url}..')
r = requests.get(url)
r.raise_for_status()
#logger.debug('Parsing JSON..')

In [7]:
url

'http://api.opennem.org.au/stats/power/network/fueltech/NEM/NSW1?month=2021-09-01'

In [8]:
raw_json = r.json()
raw_meta = {x:raw_json[x] for x in raw_json if x!='data'}
raw_data = raw_json['data']

In [9]:
raw_meta

{'type': 'power',
 'version': '3.7.8',
 'network': 'nem',
 'code': 'NSW1',
 'region': 'NSW1',
 'created_at': '2021-09-02T13:16:35+10:00'}

# Parse data

In [10]:
## Check response structure
for i in range(len(raw_data)):
    if(i < (len(raw_data)-1) ):
        assert(raw_data[i]['type']=='power')
    else:
        assert(raw_data[i]['type']=='price')

# Price data

In [11]:
price = raw_data[len(raw_data)-1]
price_meta = {x:price[x] for x in price if x!='history'}
price_data = price['history'] 
price_df = pd.DataFrame(columns = ["price"])
price_df['price'] = price_data['data']
price_df.index = pd.date_range(
    start=dateutil.parser.parse(price_data['start']), 
    end=dateutil.parser.parse(price_data['last']), 
    periods=len(price_data['data'])
    )
print(price_meta)
price_df

{'id': 'nem.nsw1.nsw1.price', 'type': 'price', 'network': 'nem', 'region': 'NSW1', 'data_type': 'price', 'code': 'NSW1', 'units': 'AUD/MWh'}


,price
2021-09-01 00:00:00+10:00,71.68
2021-09-01 00:30:00+10:00,70.99
2021-09-01 01:00:00+10:00,68.64
2021-09-01 01:30:00+10:00,65.32
2021-09-01 02:00:00+10:00,62.69
...,...
2021-09-30 21:30:00+10:00,0.00
2021-09-30 22:00:00+10:00,0.00
2021-09-30 22:30:00+10:00,0.00
2021-09-30 23:00:00+10:00,0.00


# Power data

In [12]:
power = {}
for i in range(len(raw_data)-1):
    power_data = raw_data[i]
    power_data_meta = {x:power_data[x] for x in power_data if x!='history'}
    power_data_df =pd.DataFrame(columns = [power_data_meta['code']])
    power_data_df[power_data_meta['code']] = power_data['history']['data']
    power_data_df.index = pd.date_range(
    start=dateutil.parser.parse(power_data['history']['start']), 
    end=dateutil.parser.parse(power_data['history']['last']), 
    periods=len(power_data['history']['data'])
    ) 
    power[power_data_meta['code']] = {"meta":power_data_meta,"data":power_data_df}

# Connect to database

In [13]:
with open(db_pword_file) as f:
    db_pword = f.read()

In [14]:
dbc = psycopg2.connect(user=db_user,
                        password=db_pword,
                        host="127.0.0.1",
                        port="5432",
                        database=db_name)

In [41]:
# dbc.close()

NameError: name 'dbc' is not defined

## Example SQL transaction

In [18]:
cur = dbc.cursor()
cur.execute('SELECT version()')
version = cur.fetchone()[0]
dbc.commit()


In [19]:
print(version)

PostgreSQL 13.4, compiled by Visual C++ build 1914, 64-bit


# Construct db entry

## Dim_Network

In [44]:
network = raw_meta["network"].upper()
network_longname = NETWORK_LONG_NAMES[network]

dim_network_sql = f"INSERT INTO Dim_Network(code, long_name) VALUES ('{network}', '{network_longname}')"

"INSERT INTO Dim_Network(code, long_name) VALUES ('NEM', 'National Energy Market')"

In [45]:
dim_network_sql

"INSERT INTO Dim_Network(code, long_name) VALUES ('NEM', 'National Energy Market')"

In [47]:
cur = dbc.cursor()
cur.execute(dim_network_sql)
dbc.commit()

UniqueViolation: duplicate key value violates unique constraint "dim_network_code_key"
DETAIL:  Key (code)=(NEM) already exists.


## Dim_Region

In [26]:
region_code = raw_meta["code"].upper()
region_zone = REGION_TO_ZONE_KEY[region_code]
region_shortname = raw_meta["region"].upper()
region_longname = REGION_LONG_NAMES[region_code]

dim_region_sql = f"INSERT INTO Dim_Region(code, zone, short_name, long_name) VALUES ('{region_code}', '{region_zone}', '{region_shortname}', '{region_longname}')"

In [27]:
dim_region_sql

"INSERT INTO Dim_Region(code, zone, short_name, long_name) VALUES ('NSW1', 'AUS-NSW', 'NSW1', 'New South Wales')"

In [28]:
cur = dbc.cursor()
cur.execute(dim_region_sql)
dbc.commit()

## Dim_Fueltech

In [40]:
cur = dbc.cursor()
for ft in power.keys():
    dim_fueltech_sql = f"INSERT INTO Dim_Fueltech(long_name) VALUES ('{ft}')"
    print(dim_fueltech_sql)
    cur.execute(dim_fueltech_sql)
dbc.commit()

INSERT INTO Dim_Fueltech(long_name) VALUES ('distillate')
INSERT INTO Dim_Fueltech(long_name) VALUES ('solar_utility')
INSERT INTO Dim_Fueltech(long_name) VALUES ('gas_ccgt')
INSERT INTO Dim_Fueltech(long_name) VALUES ('hydro')
INSERT INTO Dim_Fueltech(long_name) VALUES ('coal_black')
INSERT INTO Dim_Fueltech(long_name) VALUES ('wind')
INSERT INTO Dim_Fueltech(long_name) VALUES ('pumps')


## Dim_Batch

In [39]:
raw_data

[{'id': 'au.nem.nsw1.fuel_tech.distillate.power',
  'type': 'power',
  'fuel_tech': 'distillate',
  'network': 'nem',
  'region': 'NSW1',
  'data_type': 'power',
  'code': 'distillate',
  'units': 'MW',
  'history': {'start': '2021-09-01T00:00:00+10:00',
   'last': '2021-09-30T23:55:00+10:00',
   'interval': '5m',
   'data': [-0.17,
    -0.17,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1614,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1568,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1579,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1466,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1432,
    -0.1377,
    -0.1377,
    -0.1377,
    -0.1377,
    -0.1377,
    

In [38]:
power['coal_black']["data"]

,coal_black
2021-09-01 00:00:00+10:00,6831
2021-09-01 00:05:00+10:00,6803
2021-09-01 00:10:00+10:00,6841
2021-09-01 00:15:00+10:00,6837
2021-09-01 00:20:00+10:00,6864
...,...
2021-09-30 23:35:00+10:00,0
2021-09-30 23:40:00+10:00,0
2021-09-30 23:45:00+10:00,0
2021-09-30 23:50:00+10:00,0


In [15]:
for pk in power.keys():
    print(power[pk]["meta"])

{'id': 'au.nem.nsw1.fuel_tech.distillate.power', 'type': 'power', 'fuel_tech': 'distillate', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'distillate', 'units': 'MW', 'x_capacity_at_present': 178.0}
{'id': 'au.nem.nsw1.fuel_tech.solar_utility.power', 'type': 'power', 'fuel_tech': 'solar_utility', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'solar_utility', 'units': 'MW', 'x_capacity_at_present': 2747.992}
{'id': 'au.nem.nsw1.fuel_tech.gas_ccgt.power', 'type': 'power', 'fuel_tech': 'gas_ccgt', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'gas_ccgt', 'units': 'MW', 'x_capacity_at_present': 479.0}
{'id': 'au.nem.nsw1.fuel_tech.hydro.power', 'type': 'power', 'fuel_tech': 'hydro', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'hydro', 'units': 'MW', 'x_capacity_at_present': 4208.0}
{'id': 'au.nem.nsw1.fuel_tech.coal_black.power', 'type': 'power', 'fuel_tech': 'coal_black', 'network': 'nem', 'region': 'NSW1',

In [35]:
power_attribute_check('units',power)

False

In [40]:
raw_meta

{'type': 'power',
 'version': '3.7.8',
 'network': 'nem',
 'code': 'NSW1',
 'region': 'NSW1',
 'created_at': '2021-09-02T11:01:13+10:00'}

In [ ]:
## RAW
network_code = raw_meta["network"].upper()
region_code = raw_meta["region"].upper()
month_start = price_data['start']
api_version = raw_meta["version"]
pull_date = raw_meta["created_at"]

In [ ]:
## PRICE
price_interval = price_data['interval']
price_units = price_meta["units"]

In [ ]:
## POWER
def power_attribute_check(attribute,power):
    power_attributes = [power[pk]["meta"][attribute] for pk in power.keys()]
    if all(x == power_attributes[0] for x in power_attributes):
        return power_attributes[0]
    return False 



In [26]:


power_interval = 

power_units


{'type': 'power', 'version': '3.7.8', 'network': 'nem', 'code': 'NSW1', 'region': 'NSW1', 'created_at': '2021-09-01T12:24:00+10:00'}
{'id': 'nem.nsw1.nsw1.price', 'type': 'price', 'network': 'nem', 'region': 'NSW1', 'data_type': 'price', 'code': 'NSW1', 'units': 'AUD/MWh'}
distillate
{'id': 'au.nem.nsw1.fuel_tech.distillate.power', 'type': 'power', 'fuel_tech': 'distillate', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'distillate', 'units': 'MW', 'x_capacity_at_present': 178.0}
solar_utility
{'id': 'au.nem.nsw1.fuel_tech.solar_utility.power', 'type': 'power', 'fuel_tech': 'solar_utility', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'solar_utility', 'units': 'MW', 'x_capacity_at_present': 2747.992}
gas_ccgt
{'id': 'au.nem.nsw1.fuel_tech.gas_ccgt.power', 'type': 'power', 'fuel_tech': 'gas_ccgt', 'network': 'nem', 'region': 'NSW1', 'data_type': 'power', 'code': 'gas_ccgt', 'units': 'MW', 'x_capacity_at_present': 479.0}
hydro
{'id': 'au.nem.nsw1.

In [ ]:
## Dim_Fueltech 

In [ ]:
## Dim_Network 

In [ ]:
##Dim_Region 

In [ ]:
##Fact_Power 

In [ ]:
##Fact_Price 

In [28]:
##Fact_X_Cap 
for p in power:
    print(p)
    print(power[p][0]['x_capacity_at_present'])

distillate
178.0
solar_utility
2747.992
gas_ccgt
479.0
hydro
4208.0
coal_black
11990.0
wind
2125.69
pumps
840.0


,batch_id,fueltech_id,x_capacity


In [3]:
cursor = connection.cursor()

In [14]:
Dim_Batch = pd.DataFrame(columns=
    ["id",
    "network_id",
    "region_id",
    "month_start",
    "api_version",
    "pull_date",
    "interval_length",
    "price_units",
    "power_units"])

Dim_Fueltech = pd.DataFrame(columns=
    ["id",
    "long_name",
    "renewable"
    ])

Dim_Network = pd.DataFrame(columns=
        ["id",
        "code",
        "long_name"
    ])

Dim_Region = pd.DataFrame(columns=
        ["id",
        "code",
        "long_name"
    ])

Fact_Power = pd.DataFrame(columns=
        ["batch_id",
        "interval_start",
        "fueltech_id",
        "power"
    ])

Fact_Price  = pd.DataFrame(columns=
        ["batch_id",
        "interval_start",
        "price"
    ])

Fact_X_Cap  = pd.DataFrame(columns=
        ["batch_id",
        "fueltech_id",
        "x_capacity"
    ])